In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import streamlit as st

In [3]:
input_file = "CSV_filer\imdb_movies.csv"

con = sqlite3.connect('IMDB.db')
df = pd.read_csv(r'C:\Users\theod\OneDrive\Skrivbord\Tuc\VSCode\CSV_filer\imdb_movies.csv', index_col=False)
df.to_sql("IMDB", con, if_exists='replace')

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\theod\AppData\Local\Temp\ipykernel_3516\3007061487.py:1: SyntaxWarning: invalid escape sequence '\i'
  input_file = "CSV_filer\imdb_movies.csv"


10178

In [4]:
# 10 haihgest and lowest Budget 
query_budget_asc = '''SELECT names, budget_x, genre FROM IMDB WHERE genre = 'Action' ORDER BY budget_x ASC LIMIT 10;'''
query_budget_desc = '''SELECT names, budget_x, genre FROM IMDB WHERE genre = 'Action' ORDER BY budget_x DESC LIMIT 10;'''

df_action_budget_acs_10 = pd.read_sql(query_budget_asc, con)
df_action_budget_desc_10 = pd.read_sql(query_budget_desc, con)

df_action_budget_acs_desc_20 = pd.concat([df_action_budget_acs_10, df_action_budget_desc_10])

# 10 haihgest and lowest revenue 
query_revenue_asc_10 = '''SELECT names, revenue, genre FROM IMDB WHERE genre = 'Action' ORDER BY revenue ASC LIMIT 10;'''
query_revenue_desc_10 = '''SELECT names, revenue, genre FROM IMDB WHERE genre = 'Action' ORDER BY revenue DESC LIMIT 10;'''

df_action_revenue_asc_10 = pd.read_sql(query_revenue_asc_10, con)
df_action_revenue_desc_10 = pd.read_sql(query_revenue_desc_10, con)

df_action_revenue_asc_desc_20 = pd.concat([df_action_revenue_asc_10, df_action_revenue_desc_10])

# Do not feel the need for a grid when working with columns
plt.style.use('_mpl-gallery-nogrid')

fig, ax = plt.subplots(figsize=(9, 6))

# Vertical columns looked better in my opinion because it looks cleaner and wanting them on top of echother is yet again a estetic feature
df_action_budget_acs_desc_20.plot(kind='barh', x='names', y='budget_x', ax=ax, color='skyblue', label='Budget')
df_action_revenue_asc_desc_20.plot(kind='barh', x='names', y='revenue', ax=ax, color='orange', alpha=0.6, label='Revenue')

#logarithmic scale got me thinking that this is the best solution to get a somewhat reletvie persective 
ax.set_xscale('log')

ax.set_xlabel('Diffrence between earned and lost')
ax.set_ylabel('Action movies')
ax.set_title('Haighest and lowest budget and revenue per movie')

#just as a safty precaution so that the columns dont collide
plt.tight_layout()

plt.show()

In [5]:
query_country_movie_count = '''SELECT country, COUNT(*) FROM IMDB GROUP BY country ORDER BY names ASC;'''
df_country_movie_count = pd.read_sql(query_country_movie_count, con)
#print(df_country_movie_count.dtypes)

# Problem med konverteringen
#Löste konvertringen genom att separat definera värderna i min df och genom att separera dem så fick paj digaremt dem numeriska värden som krävdes

plt.style.use('_mpl-gallery-nogrid')

#The solution to my convertion dilemma
counts = df_country_movie_count['COUNT(*)'].values
labels = df_country_movie_count['country'].values

# Thuoght an index in the corner would help with the visual understanding of the pie diagram
total = sum(counts)
percentages = [count / total * 100 for count in counts]
threshold = 1.0

filtered_counts = [count for count, percentage in zip(counts, percentages) if percentage > threshold]
filtered_labels = [label for label, percentage in zip(labels, percentages) if percentage > threshold]
filtered_percentages = [percentage for percentage in percentages if percentage > threshold]

labels = [f"{label} ({percentage:.1f}%)" for label, percentage in zip(filtered_labels, filtered_percentages)]
colors = plt.get_cmap('Blues')(np.linspace(0.4, 0.8, len(filtered_counts)))
fig, ax = plt.subplots()

wedges, texts, autotexts = ax.pie(filtered_counts, labels=labels, colors=colors, radius=7, center=(4, 4),
    wedgeprops={"linewidth": 1, "edgecolor": "white"}, autopct='%1.1f%%', pctdistance=0.85)

ax.legend(wedges, labels, title="Countries", loc="upper left", bbox_to_anchor=(1, 0, 0.0, 2))
ax.set(xlim=(0, 8), xticks=np.arange(1, 8), ylim=(0, 8), yticks=np.arange(1, 8))

plt.show()

In [6]:
query_best_worst_investment = '''SELECT names, revenue, budget_x, country FROM IMDB GROUP BY NAMES'''
df_best_worst_investment = pd.read_sql(query_best_worst_investment, con)
#df_best_worst_investment

df_best_worst_investment['difference'] = df_best_worst_investment['revenue'] - df_best_worst_investment['budget_x']
df_best_worst_investment = df_best_worst_investment[['names','difference', 'country']]

specific_country = 'HK'
df_country = df_best_worst_investment[df_best_worst_investment['country'] == specific_country]

max_difference = df_country.loc[df_country['difference'].idxmax()]                          
min_difference = df_country.loc[df_country['difference'].idxmin()]

print(max_difference)
print(min_difference)

In [7]:
query_all_profit_all_movies = '''SELECT budget_x, revenue, names, country, date_x FROM IMDB GROUP BY country'''
df_all_profit_all_movies = pd.read_sql(query_all_profit_all_movies, con)

df_all_profit_all_movies['difference'] = df_all_profit_all_movies['revenue'] - df_all_profit_all_movies['budget_x']

df_profit = df_all_profit_all_movies[['names', 'difference', 'country', 'date_x']]

specific_country = 'US'
df_country = df_profit[df_profit['country'] == specific_country]

total_difference = df_country['difference'].sum()

print(f"Total profit/loss for {specific_country}: {total_difference:.2f}")


In [8]:
# Movie count for each country and avrage score on those movies 
query_movie_count_countreis = '''SELECT country, COUNT(names) AS movie_count, AVG(score) AS avg_score FROM IMDB GROUP BY country'''
df_movie_count_countreis = pd.read_sql(query_movie_count_countreis, con)

st.title("Movie Count and avrage score by countr")
st.dataframe(df_movie_count_countreis)

country_list = df_movie_count_countreis['country'].unique()
country1 = st.selectbox("Select first country: ", country_list)
country2 = st.selectbox("Select second country to compare with: ", country_list)

filtered_data1 = df_movie_count_countreis[df_movie_count_countreis['country'] == country1]
filtered_data2 = df_movie_count_countreis[df_movie_count_countreis['country'] == country2]

if not filtered_data1.empty:
    movie_count1 = filtered_data1['movie_count'].values[0]
    avg_score1 = filtered_data1['avg_score'].values[0]
    st.write(f"Country: {country1}, Number of Movies: {movie_count1}, Average Score: {avg_score1:.2f}")
else:
    st.write("No data available for the selected country.")
    
if not filtered_data2.empty:
    movie_count2 = filtered_data2['movie_count'].values[0]
    avg_score2 = filtered_data2['avg_score'].values[0]
    st.write(f"Country: {country2}, Number of Movies: {movie_count2}, Average Score: {avg_score2:.2f}")
else:
    st.write("No data available for the selected country.")



In [9]:
query_actor = "SELECT crew FROM IMDB"
df = pd.read_sql(query_actor, con)

query_movies_by_year = "SELECT names AS Movies, date_x FROM IMDB"
df_movies_by_year = pd.read_sql(query_movies_by_year, con)

df_actors = pd.DataFrame(df)

df_actors['crew_split'] = df_actors['crew'].str.split(', ')

df_exploded = df_actors.explode('crew_split').reset_index(drop=True)

df_exploded['group'] = df_exploded.index % 2
actors = df_exploded[df_exploded['group'] == 0].reset_index(drop=True)
characters = df_exploded[df_exploded['group'] == 1].reset_index(drop=True)

df_actros_and_characters = pd.DataFrame({'Actor': actors['crew_split'], 'Character': characters['crew_split']})

df_movies_by_year[['day', 'month', 'year']] = df_movies_by_year['date_x'].str.split('/', n=2, expand=True)

df_actros_and_characters_Movies_dates = df_actros_and_characters, df_movies_by_year[['year','Movies']]
df_actros_and_characters_Movies_dates



In [10]:
query_movies_by_year = "SELECT names AS Movies, date_x, country AS Country FROM IMDB"
df_movies_by_year = pd.read_sql(query_movies_by_year, con)

df_movies_by_year[['day', 'month', 'year']] = df_movies_by_year['date_x'].str.split('/', n=2, expand=True)
df_movies_by_year = df_movies_by_year[['Movies', 'year', 'Country']]

countries = df_movies_by_year['Country'].unique()
specific_country = st.selectbox("Select a country:", countries)
df_country = df_movies_by_year[df_movies_by_year['Country'] == specific_country]

df_movies_by_year

2025-01-26 10:36:55.938 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-26 10:36:56.382 
  command:

    streamlit run c:\Users\theod\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


,Movies,year,Country
0,Creed III,2023,AU
1,Avatar: The Way of Water,2022,AU
2,The Super Mario Bros. Movie,2023,AU
3,Mummies,2023,AU
4,Supercell,2023,US
...,...,...,...
10173,20th Century Women,2016,US
10174,Delta Force 2: The Colombian Connection,1990,US
10175,The Russia House,1990,US
10176,Darkman II: The Return of Durant,1995,US


In [11]:
# Analyze the most popular genres in each country based on average rating
query_popular_genres_by_country = '''
SELECT country, genre, AVG(score) AS avg_score
FROM IMDB
GROUP BY country, genre
ORDER BY avg_score DESC
'''
df_popular_genres_by_country = pd.read_sql(query_popular_genres_by_country, con)
 
selected_country = st.selectbox("Select a country:", df_popular_genres_by_country['country'].unique())
 
df_country_genres = df_popular_genres_by_country[df_popular_genres_by_country['country'] == selected_country]
 
bar_chart = alt.Chart(df_country_genres).mark_bar(color='orange').encode(
    x=alt.X('avg_score:Q', title='Average Score'),
    y=alt.Y('genre:N', title='Genre', sort='-x'),
    tooltip=['genre', 'avg_score']
).properties(title=f"Popular Genres in {selected_country}")
 
st.altair_chart(bar_chart, use_container_width=True)

In [12]:
# 2. Hämta data
query_actor = '''SELECT crew FROM IMDB'''
df_crew = pd.read_sql(query_actor, con)

query_movies_and_years = '''SELECT names AS Movies, date_x FROM IMDB'''
df_movies_and_years = pd.read_sql(query_movies_and_years, con)

# 3. Dela upp Actors och Characters från Crew
df_actors = pd.DataFrame(df_crew)
df_actors['crew_split'] = df_actors['crew'].str.split(', ')
df_exploded = df_actors.explode('crew_split').reset_index(drop=True)

# Varannan rad är Actor respektive Character
df_exploded['group'] = df_exploded.index % 2  # varannan rad för skådespelare och karaktär
actors = df_exploded[df_exploded['group'] == 0].reset_index(drop=True)  # Skådespelare
characters = df_exploded[df_exploded['group'] == 1].reset_index(drop=True)  # Karaktärer

df_actors_and_characters = pd.DataFrame({
    'Actor': actors['crew_split'], 
    'Character': characters['crew_split']
})

# 4. Förbered Movies och Year (vi säkerställer att vi har rätt format)
df_movies_and_years[['day', 'month', 'Year']] = df_movies_and_years['date_x'].str.split('/', n=2, expand=True)
df_movies_and_years = df_movies_and_years[['Movies', 'Year']].drop_duplicates()

# Debug: Skriv ut några exempel på data innan merge för att säkerställa att värdena är korrekta
st.write("Movies and Years (Preview):")
st.write(df_movies_and_years.head())

st.write("Actors and Characters (Preview):")
st.write(df_actors_and_characters.head())

# 5. Nu kopplar vi ihop skådespelare, karaktärer, filmer och år
# Vi gör en merge för att försäkra oss om att varje film får rätt information
df_combined = pd.merge(
    df_actors_and_characters,
    df_movies_and_years,
    how='left',  # Vi vill koppla varje rad från actors_and_characters till rätt film (kan vara duplicerat)
    left_on='Actor',  # Vi kopplar skådespelare till film via filmens namn
    right_on='Movies'
)

# Debug: Kontrollera om datan ser korrekt ut efter merge
st.write("Combined Data After Merge (Preview):")
st.write(df_combined.head())

# 6. Kontrollera om det finns några NaN-värden och visa dem
nan_values = df_combined[df_combined.isna().any(axis=1)]
if not nan_values.empty:
    st.write("Rows with NaN values (Check this data):")
    st.write(nan_values)

# 7. Streamlit-app för att visa data
st.title("Actor Dashboard")

# Skapa en lista över unika skådespelare
actor_list = df_combined['Actor'].unique()
specific_actor = st.selectbox("Select an Actor:", actor_list)

if specific_actor:
    # Filtrera data för vald skådespelare
    filtered_data = df_combined[df_combined['Actor'] == specific_actor]

    if not filtered_data.empty:
        st.subheader(f"Details for {specific_actor}")
        st.write(filtered_data[['Actor', 'Character', 'Movies', 'Year']])
    else:
        st.write(f"No data found for {specific_actor}.")
else:
    st.write("Please select an actor from the dropdown.")

In [13]:
# kolla på villken skådespelare som förekommer mest (top 10) 
# Den mest lucrativa filmen kontra filmen som tjänade minst från respektive land (kanske kör 25 länder)
# Den största filmen kontra största floppen på rating för respektive land (kanske kör 25 länder) 
# tex filmer från usa genom åren hur mycket dem tjänat(ecdf(x))
# Det dominerande landet i filmindustrin, baserat på mängden filmer som dem släpps och score dem fått.
# kan kanske fixa en combo på dem ovan att paj diagreamet är interaktivt? Skulle vara svin cool och lite HTML vibe
# tack vara mitt paj-diagram skulle man kunna ta dem 3 eller 5 största länder kolla hur mycket dem tjänat genom åren och sen använda st.line_chart
# Hur mycket ett land har tjänat och förlotar totalt på alla filmer och hur många filmer dem har släppt
# Kopplat till den ovan, om usa har släppt 20 filmer och tjänat 20kr och japan släppt 3 filmer och tjänat 40kr 
# En sorteringsfunction

# Ändra status på koden så att filmer som inte släppts än inte ska inkluderas